In [1]:
import numpy as np

In [2]:
with open("Total_movie.tsv") as tsvfile:
    total_file = tsvfile.read().split("\n")

In [3]:
print(total_file[0])

tconst	 titleType	 primaryTitle	 isAdult	 startYear	 endYear	 runtimeMinutes	 genres	 region	 language	IMDB rating


In [4]:
"""
Codes used to generate catagorical IDs, should not be run again as the ID has already been genrated
"""
# genreList = []
# for line in total_file[1:]:
#     if len(line) > 1:
#         genres = line.split("\t")[7].split(",")
#         for g in genres:
#             g = g.replace("\"","")
#             if not (g in genreList):
#                 genreList.append(g)
# genreList.remove("\\N")
# genreList = ["\\N"] + genreList
# print(len(genreList))
# with open("Genre_ID.tsv", "w+") as f:
#     for i, item in enumerate(genreList):
#         f.write(f"{i-1}\t{item}\n")
# regionList = []
# for line in total_file[1:]:
#     if len(line) > 1:
#         regions = line.split("\t")[8].split(",")
#         for r in regions:
#             r = r.replace("\"","")
#             if not (r in regionList):
#                 regionList.append(r)
# regionList.remove("\\N")
# regionList = ["\\N"] + regionList
# print(len(regionList))
# with open("Region_ID.tsv", "w+") as f:
#     for i, item in enumerate(regionList):
#         f.write(f"{i-1}\t{item}\n")

# languageList = []
# for line in total_file[1:]:
#     if len(line) > 1:
#         languages = line.split("\t")[9].split(",")
#         for l in languages:
#             l = l.replace("\"","")
#             if not (l in languageList):
#                 languageList.append(l)
# languageList.remove("\\N")
# languageList = ["\\N"] + languageList
# print(len(languageList))
# with open("Language_ID.tsv", "w+") as f:
#     for i, item in enumerate(languageList):
#         f.write(f"{i-1}\t{item}\n")

'\nCodes used to generate catagorical IDs, should not be run again as the ID has already been genrated\n'

In [5]:
with open("Genre_ID.tsv", "r") as f:
    genreList = []
    for line in f.read().split("\n")[:-1]:
        genreList.append(line.split("\t")[1])
with open("Language_ID.tsv", "r") as f:
    languageList = []
    for line in f.read().split("\n")[:-1]:
        languageList.append(line.split("\t")[1])
with open("Region_ID.tsv", "r") as f:
    regionList = []
    for line in f.read().split("\n")[:-1]:
        regionList.append(line.split("\t")[1])

In [6]:
class Movie():
    def __init__(self, attributes, genreList, languageList, regionList):
        self.ID = None
        self.title = attributes[2]
        self.isAdult = int(attributes[3])
        self.startYear = self.get_num(attributes[4])
        self.endYear = self.get_num(attributes[5])
        self.length = self.get_num(attributes[6])
        self.genres = self.get_cat(attributes[7], genreList)
        self.region = self.get_cat(attributes[8], regionList)[0]
        self.language = self.get_cat(attributes[9], languageList)[0]
        self.rating = self.get_num(attributes[10])
    
    def get_cat(self, attribute:str, aList:list):
        myList = []
        for a in attribute.replace("\"","").split(","):
            index = aList.index(a) - 1
            myList.append(index)
        return myList
    
    def get_num(self, attribute:str):
        try:
            attribute = float(attribute)
        except:
            attribute = -1.
        return attribute

In [7]:
all_movie = []
i = 0
for line in total_file[1:-1]:
    m = Movie(line.split("\t"), genreList, languageList, regionList)
    if m.rating != "\\N":
        m.ID = i
        i +=1
        all_movie.append(m)

In [8]:
with open("ID_Title.tsv", "w+") as f:
    f.write("ID\tTitle\n")
    for m in all_movie:
        f.write(str(m.ID) + "\t" + m.title + "\n")

In [9]:
with open("ID_Rating.tsv", "w+") as f:
    f.write("ID\tRating\n")
    for m in all_movie:
        f.write(str(m.ID) + "\t" + str(m.rating) + "\n")

In [11]:
with open("ID_Attributes.tsv", "w+") as f:
    f.write("ID\tisAdult\tstartYear\tendYear\tlength\tgenres\tregion\tlanguage\n")
    for m in all_movie:
        f.write(str(m.ID) + "\t" + str(m.isAdult) + "\t" + str(m.startYear) + "\t" + str(m.endYear) + "\t" + str(m.length) + "\t" + str(m.genres).replace("[","").replace("]","")  + "\t" + str(m.region)  + "\t" + str(m.language) +"\n")

In [21]:
npAttributes = np.zeros([len(all_movie),10])
def expand(mIn, size):
    small = len(mIn)
    assert small <= size
    r = [-1 for _ in range(size)]
    r[:small] = mIn 
    return r

for i, m in enumerate(all_movie):
    npAttributes[i,:5] = [m.ID, m.isAdult,m.startYear,m.endYear,m.length]
    npAttributes[i,5:8] = expand(m.genres,3)
    npAttributes[i,8:] = [m.region, m.language]
np.save("npAttributes",npAttributes)

In [22]:
ratings = []
for i, m in enumerate(all_movie):
    ratings.append(m.rating)
np.save("npRatings", ratings)